Question (1)

python subroutine to solve for the inverse position kinematics for the Stanford manipulator

In [2]:

import math
import numpy as np

# Define the known parameters
d1 = 1.0  # Replace with your actual d1 value
a2 = 2.0  # Replace with your actual a2 value

# Function for inverse kinematics
def stanford_manipulator(x, y, z):
    # r is already calculated in your code, so we don't need to calculate it again
    r = x**2 + y**2

    s = z - d1
    theta1 = np.pi + np.arctan(x / y)

    theta2 = np.pi - np.arctan(r / s)

    d3 = math.sqrt(r**2 + s**2) - a2

    return theta1, theta2, d3

# Numerical values for end-effector position
x = 3.0
y = 2.0
z = 4.0

# Compute the joint variables
theta1, theta2, d3 = stanford_manipulator(x, y, z)

# Print the results
print("Theta1:", theta1)
print("Theta2:", theta2)
print("d3:", d3)



Theta1: 4.124386376837122
Theta2: 1.7975951748487824
d3: 11.341664064126334


Question (2)

python subroutine for the joint velocities using end-effector cartesian velocities.

In [3]:
import numpy as np

def joint_velocities(Jacobian, end_effector_velocity):

    Inverse_Jacobian = np.linalg.pinv(Jacobian)

    joint_velocities = np.dot(Inverse_Jacobian, end_effector_velocity)

    return joint_velocities

# Example values for Jacobian and end-effector velocity
Jacobian = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

end_effector_velocity = np.array([1, 2, 3])

# Compute joint velocities
resulting_joint_velocities = joint_velocities(Jacobian, end_effector_velocity)

print("Joint Velocities:", resulting_joint_velocities)




Joint Velocities: [1. 2. 3.]


Question(5)

Python subroutine for the inverse kinematics of the spherical wrist

In [4]:
import math

def spherical_wrist_inverse_kinematics(x, y, z, link_lengths):

    #link lengths
    l1, l2, l3 = link_lengths

    #first joint angle (q1)
    q1 = math.atan2(y, x)

    # distance from the base to the projection of the end-effector in the xy-plane
    D_xy = math.sqrt(x**2 + y**2)

    #height difference between the end-effector and the first wrist joint
    delta_z = z - l1

    #distance between the first wrist joint and the end-effector in the xy-plane
    D = math.sqrt(D_xy**2 + delta_z**2)

    #angles required to form a right triangle with sides l2 and l3
    alpha = math.acos((l2**2 + l3**2 - D**2) / (2 * l2 * l3))
    beta = math.atan2(delta_z, D_xy)

    #second joint angle (q2)
    q2 = math.pi/2 - alpha - beta

    #third joint angle (q3)
    q3 = math.atan2(l3 * math.sin(alpha), l2 + l3 * math.cos(alpha))

    return q1, q2, q3


question (6)a.

Write a code incorporating dynamics of the 3 DOF robot. Further,
simulate it with small initial conditions and small constant torque
values to observe the dynamic behavior (important to keep the
torques small, else the robot may be continuously accelerating).


In [5]:
import numpy as np
import matplotlib.pyplot as plt

# Define robot parameters (mass, inertia, etc.)
# Define your DH parameters and other dynamic properties here.

# Simulation parameters
dt = 0.01  # Time step
num_steps = 1000  # Number of simulation steps
small_torque = 0.1  # Small constant torque value

q = np.zeros(3)  # Joint angles
qd = np.zeros(3)  # Joint velocities


joint_angle_data = []
joint_torque_data = []

# Simulation loop
for step in range(num_steps):
    # Calculate joint accelerations using the equations of motion
    # Replace the following with your actual dynamics model
    qdd = np.sin(q)  # A simplified dynamics model

    # Numerical integration to update joint velocities and positions
    qd = qd + qdd * dt
    q = q + qd * dt

    # Apply small torque values (use your control strategy here)
    # For demonstration, a simple constant torque is applied to the first joint.
    tau = np.array([small_torque, 0, 0])

    # Store data for analysis
    joint_angle_data.append(q.copy())
    joint_torque_data.append(tau.copy())

# Convert data lists to numpy arrays for easier analysis or plotting
joint_angle_data = np.array(joint_angle_data)
joint_torque_data = np.array(joint_torque_data)

# Plot joint angles and torques
time_steps = np.arange(num_steps) * dt
plt.figure(figsize=(12, 6))
for i in range(3):
    plt.subplot(3, 1, i+1)
    plt.plot(time_steps, joint_angle_data[:, i], label=f'Joint {i+1} Angle')
    plt.ylabel(f'Joint {i+1} Angle (rad)')
    plt.grid()
plt.xlabel('Time (s)')
plt.show()

plt.figure(figsize=(12, 6))
for i in range(3):
    plt.subplot(3, 1, i+1)
    plt.plot(time_steps, joint_torque_data[:, i], label=f'Joint {i+1} Torque')
    plt.ylabel(f'Joint {i+1} Torque (Nm)')
    plt.grid()
plt.xlabel('Time (s)')
plt.show()





question(6)b.

Code all four versions of Independent joint control (simple, slightly
more sophisticated, feedforward, computed torque), and implement
all four versions on the dynamics code above. Pick a relatively simple
desired trajectory and some small stochastic disturbance (to be
added to the code), and test and compare the performance of all
four independent joint control methods. Document the results in
terms of plots and comparisons.

In [ ]:
import numpy as np

# Define robot parameters, simulation parameters, initial conditions, etc.

# Define control gains for PD controllers (for methods 1 and 2)
Kp = np.array([1.0, 1.0, 1.0])  # Proportional gains
Kd = np.array([0.1, 0.1, 0.1])  # Derivative gains

desired_trajectory = np.array([np.linspace(0, 2 * np.pi, num_steps)])

disturbance = np.random.normal(0, 0.01, num_steps)

joint_angles = []
joint_torques = []
joint_errors = []

# Simulation loop
for step in range(num_steps):

    # Control methods
    # Method 1: Simple Independent Joint Control
    error = desired_trajectory[:, step] - q  # Error in joint angles
    control_input = Kp * error

    # Add disturbance to the control input
    control_input += disturbance[step]

    # Method 2: Slightly More Sophisticated Control (PD)
    error = desired_trajectory[:, step] - q  # Error in joint angles

    error_d = 0 - qd                     # Desired velocity is zero

    control_input = Kp * error + Kd * error_d

    # Add disturbance to the control input
    control_input += disturbance[step]

    # Method 3: Feedforward Control

    # Calculate desired torques based on desired trajectory

    desired_torques = your_feedforward_calculation(desired_trajectory[:, step], qd)

    # Add disturbance to the desired torques
    desired_torques += disturbance[step]

    # Method 4: Computed Torque Control
    # Calculate desired torques using computed torque control
    desired_torques = your_computed_torque_calculation(desired_trajectory[:, step], q, qd)

    # Add disturbance to the desired torques
    desired_torques += disturbance[step]

    # Apply control input to the robot and update the state

    # Store data for analysis (joint angles, torques, errors)

# End of simulation
